In [1]:
import numpy as np

from Related_Codes.DCNAutoencoder import DCNAutoencoder, DCNCDAutoencoder
from Related_Codes.DECAutoencoder import DECAutoencoder, DECCDAutoencoder
from Related_Codes.IDECAutoencoder import IDECAutoencoder, IDECCDAutoencoder
from General_Functions import General_Functions

from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.manifold import LocallyLinearEmbedding, SpectralEmbedding, Isomap, TSNE

import torch

import os
import sys

from Datasets.Datasets_Functions import *
from Visualization import Visualization

In [2]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

In [3]:
hashmap_path = path_to_module + 'Datasets/'
hashmap = get_hashmap(hashmap_path)
print(hashmap)
dataset_name = "emnist_balanced_letters"
dataset_properties = hashmap[dataset_name]
batch_size = dataset_properties['batch_size'] = 256
n_clusters = dataset_properties['n_clusters'] #= 4
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, dataset_properties)
print(input_dim)

{'tcga': {'batch_size': 256, 'n_clusters': 5, 'module_name': 'Datasets.Datasets'}, 'emnist_balanced_letters': {'batch_size': 256, 'n_clusters': 10, 'module_name': 'Datasets.Datasets'}, 'emnist_mnist': {'batch_size': 1024, 'n_clusters': 10, 'module_name': 'Datasets.Datasets'}, 'emnist_balanced_digits': {'batch_size': 256, 'n_clusters': 10, 'module_name': 'Datasets.Datasets'}, 'dermatology': {'batch_size': 64, 'n_clusters': 6, 'module_name': 'Datasets.Datasets'}, 'ecoil': {'batch_size': 64, 'n_clusters': 8, 'module_name': 'Datasets.Datasets'}, 'iris': {'batch_size': 64, 'n_clusters': 3, 'module_name': 'Datasets.Datasets'}, 'rings': {'batch_size': 64, 'n_clusters': 2, 'module_name': 'Datasets.Datasets'}, 'wine': {'batch_size': 64, 'n_clusters': 3, 'module_name': 'Datasets.Datasets'}, 'australian': {'batch_size': 64, 'n_clusters': 2, 'module_name': 'Datasets.Datasets'}, 'moons': {'batch_size': 64, 'n_clusters': 2, 'module_name': 'Datasets.Datasets'}, 'squeezed_gauss': {'batch_size': 64, 'n

[nltk_data] Downloading package punkt to C:\Users\PAPASOFT
[nltk_data]     INC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\PAPASOFT
[nltk_data]     INC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1


In [4]:
visualization = Visualization()

### Dcn

In [5]:

"""
parser.add_argument('--n-classes', type=int, default=10,
                    help='output dimension')
"""
n_clusters = dataset_properties['n_clusters'] #= 4

# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size'] #= 256

# Pre-Training Epochs and Learning Rate
"""
parser.add_argument('--pre-epoch', type=int, default=50, 
                    help='number of pre-train epochs')
"""
n_pret_epochs = 100
pret_lr = 5e-4

# Lamdas, Training Epochs and Learning Rate
"""
parser.add_argument('--epoch', type=int, default=100,
                    help='number of epochs to train')
"""
n_epochs = 100
"""
parser.add_argument('--lr', type=float, default=1e-4,
                    help='learning rate (default: 1e-4)')
"""
lr = 5e-4
"""
parser.add_argument('--lamda', type=float, default=1,
                    help='coefficient of the reconstruction loss')
"""
lamda = 1
"""
parser.add_argument('--beta', type=float, default=1,
                    help=('coefficient of the regularization term on '
                          'clustering'))
"""
beta = 1e-5
"""
parser.add_argument('--latent_dim', type=int, default=10,
                    help='latent space dimension')
"""
negative_slope = 0
latent_dim = 10
n_channels = 1
is_MLP_AE = False

if(is_MLP_AE):
    dcn_autoencoder = DCNAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope)
else:  
    input_dim = 1
    dcn_autoencoder = DCNCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope, n_channels=n_channels) 
dcn_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
dcn_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
dcn_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, lamda=lamda, beta=beta )
dcn_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
dcn_autoencoder.set_path()
dcn_autoencoder = dcn_autoencoder.to(device)
dcn_autoencoder
pretrain = True
if(pretrain):
    dcn_autoencoder.pretrain_autoencoder()
    dcn_autoencoder.save_pretrained_weights()
else:
    model_save_path = dcn_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    dcn_autoencoder.load_state_dict(torch.load(model_save_path))

Epoch: 0, Loss: 19.117621
Epoch: 1, Loss: 6.657898
Epoch: 2, Loss: 5.376824
Epoch: 3, Loss: 4.595087
Epoch: 4, Loss: 4.103640
Epoch: 5, Loss: 3.782311
Epoch: 6, Loss: 3.557128
Epoch: 7, Loss: 3.337781
Epoch: 8, Loss: 3.200658
Epoch: 9, Loss: 3.064718
Epoch: 10, Loss: 2.966107
Epoch: 11, Loss: 2.870135
Epoch: 12, Loss: 2.774955
Epoch: 13, Loss: 2.712431
Epoch: 14, Loss: 2.649657
Epoch: 15, Loss: 2.586892
Epoch: 16, Loss: 2.552765
Epoch: 17, Loss: 2.500370
Epoch: 18, Loss: 2.438231
Epoch: 19, Loss: 2.426848
Epoch: 20, Loss: 2.389579
Epoch: 21, Loss: 2.346014
Epoch: 22, Loss: 2.318884
Epoch: 23, Loss: 2.282825
Epoch: 24, Loss: 2.245002
Epoch: 25, Loss: 2.225906
Epoch: 26, Loss: 2.212159
Epoch: 27, Loss: 2.193102
Epoch: 28, Loss: 2.167720
Epoch: 29, Loss: 2.133860
Epoch: 30, Loss: 2.121269
Epoch: 31, Loss: 2.109725
Epoch: 32, Loss: 2.086991
Epoch: 33, Loss: 2.069735
Epoch: 34, Loss: 2.038218
Epoch: 35, Loss: 2.032554
Epoch: 36, Loss: 2.015365
Epoch: 37, Loss: 1.997417
Epoch: 38, Loss: 1.98

In [ ]:
dcn_autoencoder.train_autoencoder()
cluster_centers = dcn_autoencoder.get_cluster_centers()
_, dcn_autoencoder_reduced_data, labels = dcn_autoencoder.get_latent_data()
General_Functions().save_excel(dcn_autoencoder.data_dir_path, dcn_autoencoder.df_eval)
visualization.plot_tsne(dcn_autoencoder_reduced_data, labels, labels, cluster_centers, dcn_autoencoder.data_dir_path)

Ep: 0 L: 2.0214 Rec L: 1.5869 Dist L: 0.4345 ACC: 0.73 PUR: 0.77 NMI: 0.71 ARI: 0.61
Ep: 1 L: 1.9445 Rec L: 1.5905 Dist L: 0.3540 ACC: 0.73 PUR: 0.78 NMI: 0.72 ARI: 0.62
Ep: 2 L: 1.8851 Rec L: 1.5720 Dist L: 0.3131 ACC: 0.73 PUR: 0.78 NMI: 0.72 ARI: 0.62
Ep: 3 L: 1.8534 Rec L: 1.5700 Dist L: 0.2835 ACC: 0.74 PUR: 0.78 NMI: 0.73 ARI: 0.63
Ep: 4 L: 1.8181 Rec L: 1.5575 Dist L: 0.2607 ACC: 0.74 PUR: 0.78 NMI: 0.73 ARI: 0.63
Ep: 5 L: 1.8009 Rec L: 1.5571 Dist L: 0.2438 ACC: 0.74 PUR: 0.79 NMI: 0.73 ARI: 0.64
Ep: 6 L: 1.7971 Rec L: 1.5659 Dist L: 0.2312 ACC: 0.74 PUR: 0.79 NMI: 0.74 ARI: 0.64
Ep: 7 L: 1.7673 Rec L: 1.5481 Dist L: 0.2192 ACC: 0.74 PUR: 0.79 NMI: 0.74 ARI: 0.64
Ep: 8 L: 1.7888 Rec L: 1.5776 Dist L: 0.2112 ACC: 0.75 PUR: 0.79 NMI: 0.74 ARI: 0.64
Ep: 9 L: 1.7440 Rec L: 1.5431 Dist L: 0.2008 ACC: 0.75 PUR: 0.79 NMI: 0.74 ARI: 0.64
Ep: 10 L: 1.7394 Rec L: 1.5463 Dist L: 0.1931 ACC: 0.75 PUR: 0.79 NMI: 0.74 ARI: 0.64
Ep: 11 L: 1.7105 Rec L: 1.5254 Dist L: 0.1851 ACC: 0.75 PUR: 0.8

In [ ]:
visualization.plot(dcn_autoencoder_reduced_data, labels, labels, cluster_centers, dcn_autoencoder.data_dir_path)